In [56]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [57]:
scenarios_df = CSV.read("load_profiles.csv", DataFrame)
n_scenarios = size(scenarios_df, 2)
M = 10000
in_sample_n = 100

100

# Task 2.1

In [58]:
#define function
function also_x_loop(q)
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)

    @variable(model, cap_bid)
    @variable(model, bin_var[1:60, 1:in_sample_n])

    @constraint(model, cap_bid >= 0)


    for scenario in 1:in_sample_n
        for minute in 1:60
            @constraint(model, cap_bid - scenarios_df[minute, scenario] <= M* bin_var[minute, scenario])
            @constraint(model, bin_var[minute, scenario] <= 1)
            @constraint(model, bin_var[minute, scenario] >= 0)
        end
    end
    @constraint(model, sum(bin_var[minute, scenario] for scenario in 1:in_sample_n, minute in 1:60) <= q)

    # Define the objective function
    @objective(model, Max, cap_bid)

    # Solve the model
    optimize!(model)

    # Check the status of the solution
    status = termination_status(model)
    if status == MOI.OPTIMAL
        #println("Optimal solution found")
        #println("Optimal solution cap_bid: ", value(cap_bid))
    elseif status == MOI.INFEASIBLE
        println("No feasible solution found")
    else
        println("Solver status: ", status)
    end
    return JuMP.value.(bin_var), JuMP.value(cap_bid)
end

function also_x_function(p_value)

    stopping_parameter = 0.0001

    lwr_q = 0
    upr_q = (1-p_value)*in_sample_n*60
    counter = 0

    while upr_q - lwr_q >= stopping_parameter
        q = (lwr_q + upr_q)/2
        bin_var, cap_bid = also_x_loop(q)
        num_zeros = count(x -> x == 0, bin_var)
        #println("q: ", q, " num_zeros: ", num_zeros)

        if num_zeros/length(bin_var) > p_value
            lwr_q = q
        else
            upr_q = q
        end

        if upr_q - lwr_q < stopping_parameter
            println("q: ", q, " num_zeros: ", num_zeros)
            println("cap_bid: ", cap_bid)
            return cap_bid
        end

        if counter > 50
            println("q: ", q, " num_zeros: ", num_zeros)
            println("cap_bid: ", cap_bid)
            return cap_bid
        end
        counter += 1
    end
end

also_x_function (generic function with 1 method)

In [59]:
function cvar_function(p_value)
    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)

    @variable(model, cap_bid)
    @variable(model, beta <= 0)
    @variable(model, theta[1:60, 1:in_sample_n])

    @constraint(model, cap_bid >= 0)

    for scenario in 1:in_sample_n
        for minute in 1:60
            @constraint(model, cap_bid - scenarios_df[minute, scenario] <= theta[minute, scenario])
            @constraint(model, beta <= theta[minute, scenario])
        end
    end
    @constraint(model, (1/(60*in_sample_n))*sum(theta[minute, scenario] for scenario in 1:in_sample_n, minute in 1:60) <= p_value*beta) 

    # Define the objective function
    @objective(model, Max, cap_bid)

    # Solve the model
    optimize!(model)

    # Check the status of the solution
    status = termination_status(model)
    if status == MOI.OPTIMAL
        #println("Optimal solution found")
        println("Optimal solution cap_bid: ", value(cap_bid))
    elseif status == MOI.INFEASIBLE
        println("No feasible solution found")
    else
        println("Solver status: ", status)
    end

    return value(cap_bid)
end

cvar_function (generic function with 1 method)

In [60]:
p_value = 0.9
also_x_value = also_x_function(p_value)
cvar_value = cvar_function(p_value)

q: 1.6078233718872066 num_zeros: 5400
cap_bid: 318.36119837424263
Optimal solution cap_bid: 291.5641421761087


291.5641421761087

# Task 2.2

In [61]:
function test(p_value, cvar_value, also_x_value)
    below_cvar = 0
    below_also_x = 0

    #check how many values are above the cvar_value
    for i in 1+in_sample_n:n_scenarios

        minute_values = scenarios_df[:,i]
        #println(minute_values)
        #println(sum(minute_values .> cvar_value))
        if sum(minute_values .> cvar_value) <= p_value*length(minute_values)
            below_cvar += 1
        end
        if sum(minute_values .> also_x_value) <= p_value*length(minute_values)
            below_also_x += 1
        end
    end

    println("Allowed number of scenarios below CVaR: ", 0.15*(n_scenarios-in_sample_n))

    if below_cvar <= 0.15*(n_scenarios-in_sample_n)
        println("CVaR test passed")
        println(below_cvar)
    else
        println("CVaR test failed")
        println(below_cvar)
    end

    if below_also_x <= 0.15*(n_scenarios-in_sample_n)
        println("Also_x test passed")
        println(below_also_x)
    else
        println("Also_x test failed")
        println(below_also_x)
    end
end

test (generic function with 1 method)

In [62]:
test(p_value, also_x_value, cvar_value)

Allowed number of scenarios below CVaR: 15.0
CVaR test failed
36
Also_x test failed
20


# Task 2.3

In [63]:
p_value = 0.8
also_x_value = also_x_function(p_value)
cvar_value = cvar_function(p_value)
test(p_value, cvar_value, also_x_value)

q: 3.3803701400756827 num_zeros: 4800
cap_bid: 338.97419836569674
Optimal solution cap_bid: 310.80444719839306
Allowed number of scenarios below CVaR: 15.0
CVaR test failed
21
Also_x test failed
47


In [66]:
p_value = 0.999
also_x_value = also_x_function(p_value)
cvar_value = cvar_function(p_value)
test(p_value, cvar_value, also_x_value)

q: 0.01089477539062501 num_zeros: 5995
cap_bid: 232.6656707497823
Optimal solution cap_bid: 214.5261157338029
Allowed number of scenarios below CVaR: 15.0
CVaR test passed
7
Also_x test passed
11
